### Install the required packages

In [ ]:
# %pip install python-dotenv
%pip install --upgrade azure-ai-textanalytics

### Load the configuration settings

In [ ]:
from dotenv import dotenv_values,load_dotenv
import sys
sys.path.append('../common')
from helper import Helper

# Load the environment variables
load_dotenv()

# Load the secrets from the environment file
secrets = dotenv_values("secrets.env") # Modify environment file name to match respective environment file

# Load APIM Environment Variables
api_key = secrets.get("API_KEY")
environment = secrets.get("APIM_NAME")
account_connection_string = secrets.get("AZURE_STORAGE_CONNECTION_STRING")
region_override = secrets.get("REGION_OVERRIDE")

helper = Helper(environment, "/language", api_key, region_override)

In [ ]:
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential
from azure.core.pipeline.policies import HeadersPolicy

base_text = (
    "At Microsoft, we have been on a quest to advance AI beyond existing techniques, "
    "by taking a more holistic, human-centric approach to learning and understanding. "
    "As Chief Technology Officer of Azure AI services, I have been working with a team of amazing scientists and engineers "
    "to turn this quest into a reality. In my role, I enjoy a unique perspective in viewing the relationship among three attributes "
    "of human cognition: monolingual text (X), audio or visual sensory signals, (Y) and multilingual (Z). At the intersection of all three, "
    "there's magic—what we call XYZ-code as illustrated in Figure 1—a joint representation to create more powerful AI that can speak, hear, see, "
    "and understand humans better. We believe XYZ-code enables us to fulfill our long-term vision: cross-domain transfer learning, spanning modalities "
    "and languages. The goal is to have pretrained models that can jointly learn representations to support a broad range of downstream AI tasks, "
    "much in the way humans do today. Over the past five years, we have achieved human performance on benchmarks in conversational speech recognition, "
    "machine translation, conversational question answering, machine reading comprehension, and image captioning. These five breakthroughs provided us "
    "with strong signals toward our more ambitious aspiration to produce a leap in AI capabilities, achieving multi-sensory and multilingual learning that "
    "is closer in line with how humans learn and understand. I believe the joint XYZ-code is a foundational component of this aspiration, if grounded with "
    "external knowledge sources in the downstream AI tasks."
)

# Generate N documents
num_docs = 3
documents = [
    {
        "id": str(i),
        "language": "en",
        "text": base_text
    }
    for i in range(1, num_docs + 1)
]

api_version = "2023-04-01"
# Create the client
client = TextAnalyticsClient(
    endpoint= F"https://{environment}",
    credential= AzureKeyCredential(api_key),
    api_version= api_version,
    headers_policy=HeadersPolicy({
        "Ocp-Apim-Subscription-Key": api_key,
        "x-kpmg-charge-code": '803000017570',
        "x-kpmg-region-override": region_override,
}))

### Submit Abstractive Summarization Job

In [ ]:
# Make the call to the summarization endpoint
poller = client.begin_abstract_summary(
    documents=documents,
    language="en",
    sentence_count=1
)
result = poller.result()

assert poller.status() == "succeeded", f"Summarization job did not succeed, status: {poller.status()}"

### Get Summarization Job

In [ ]:
from IPython.display import display, Markdown

for id, doc in enumerate(result):
    for idx, summary in enumerate(doc.summaries):
        display(Markdown(f"**Abstractive Summary of doc {id+1}**: {summary.text}"))

### Submit Abstractive Summarization Job

In [ ]:
# Make the call to the summarization endpoint
poller = client.begin_extract_summary(
    documents=documents,
    language="en"
)
result = poller.result()

assert poller.status() == "succeeded", f"Summarization job did not succeed, status: {poller.status()}"

for idx, doc in enumerate(result):
    if hasattr(doc, 'sentences') and doc.sentences:
        for sentence_idx, sentence in enumerate(doc.sentences):
            display(Markdown(f"**Sentence {sentence_idx + 1}:** {sentence.text}"))

### Submit Key Phrase extraction Job

In [ ]:
# Make a call to extract key phrases from the documents
responses = client.extract_key_phrases(documents)
for idx, doc in enumerate(responses):
    if not doc.is_error:
        print(f"Document {idx+1} Key Phrases:")
        for phrase in doc.key_phrases:
            print("\t", phrase)
    else:
        print(f"Document {idx+1} error: {doc.id}, {doc.error}")

### Submit language detection job

In [ ]:
# Make a call to detect language from the documents
responses = client.detect_language(documents)
reviewed_docs = [doc for doc in responses if not doc.is_error]

for idx, doc in enumerate(reviewed_docs):
        print("Review #{} is in '{}', which has ISO639-1 name '{}'\n".format(
            idx, doc.primary_language.name, doc.primary_language.iso6391_name
        ))

### Entity Extraction

In [ ]:
# Function for recognizing entities from text
def entity_recognition_example(client):

    try:
        # The result is a list, one for each document sent
        doc_results = client.recognize_entities(documents=documents)

        # --- ASSERTION CASES ---

        # 1. Assert that the number of results matches the number of documents sent
        assert len(doc_results) == len(documents)
        print(f"Assertion Passed: Received {len(doc_results)} results for {len(documents)} documents.")

        # Process the first document's result for more detailed assertions
        result = doc_results[0]

        # 2. Assert that the API call for the document was successful (no errors)
        assert not result.is_error
        print(f"Assertion Passed: Document '{result.id}' processed successfully.")

        # 3. Assert that at least one entity was found
        assert len(result.entities) > 0
        print(f"Assertion Passed: Found {len(result.entities)} entities.")

        # 4. Find a specific entity and assert its properties
        # In the sample text, "Microsoft" should be recognized as an Organization.
        microsoft_entity = next((entity for entity in result.entities if entity.text == "Microsoft"), None)
        
        assert microsoft_entity is not None
        print(f"Assertion Passed: Found the 'Microsoft' entity.")
        
        assert microsoft_entity.category == "Organization"
        print(f"Assertion Passed: 'Microsoft' is correctly categorized as 'Organization'.")


        # --- Original Print Logic ---
        print("\nNamed Entities:\n")
        for entity in result.entities:
            print(f"\tText: \t {entity.text}",
                  f"\tCategory: \t {entity.category}",
                  f"\tSubCategory: \t {entity.subcategory}",
                  f"\n\tConfidence Score: \t {round(entity.confidence_score, 2)}",
                  f"\tLength: \t {entity.length}",
                  f"\tOffset: \t {entity.offset}\n")
    except Exception as err:
        print("Encountered exception. {}".format(err))
entity_recognition_example(client)